### Introduction
In this notebook, we will demonstrate the power and flexibility of the Evolutionary Forest Regressor, a machine learning algorithm that combines the strengths of decision trees and genetic programming, while supporting Pandas DataFrame as input. This makes it easy to work with complex data structures and perform data preprocessing efficiently. We will walk you through a simple example, primarily demonstrating how to generate a dataset, train the model, evaluate its performance, and most importantly, showcase the process of synthesizing high-order features using Pandas DataFrames as input.

### Step 1: Importing necessary libraries

First, we import all the required libraries, including Pandas, NumPy, and the EvolutionaryForestRegressor from the evolutionary_forest package.

In [10]:
import sys

sys.path.insert(0, '../')

import random
import string
import pandas as pd
import numpy as np
from sklearn.datasets import make_friedman1
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from evolutionary_forest.forest import EvolutionaryForestRegressor

### Step 2: Generating a dataset

We generate a synthetic dataset using the make_friedman1 function from the scikit-learn library, which creates a dataset with 100 samples, five features, and a single target variable. We then convert the NumPy arrays to Pandas DataFrames for better interpretability.

In [11]:
random.seed(0)
np.random.seed(0)

# Generate dataset
X, y = make_friedman1(n_samples=100, n_features=5, random_state=0)

# Convert numpy arrays to pandas dataframe
X = pd.DataFrame(X, columns=list(string.ascii_uppercase[:X.shape[1]]))
y = pd.DataFrame(y, columns=['Target'])

### Step 3: Splitting the dataset

To evaluate the performance of our model, we split the dataset into training and testing sets using the train_test_split function from scikit-learn, with 80% of the data reserved for training and the remaining 20% for testing.

In [12]:
# Split dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Step 4: Training the Evolutionary Forest

We instantiate an EvolutionaryForestRegressor with various hyperparameters, such as the maximum tree height, normalization, selection method, and base learner, among others. Next, we fit the model to the training data using the fit method.

In [13]:
# Train Evolutionary Forest
r = EvolutionaryForestRegressor(max_height=5, normalize=True, select='AutomaticLexicase',
                                gene_num=10, boost_size=100, n_gen=20, n_pop=200, cross_pb=1,
                                base_learner='Random-DT', verbose=True, n_process=1)
r.fit(x_train, y_train)

### Step 5: Evaluating the model

To evaluate the performance of our trained model, we use the r2_score metric from scikit-learn, which measures the proportion of variance in the target variable that is predictable from the input features. The closer the R² score is to 1, the better the model is at predicting the target variable.

In [14]:
print(r2_score(y_test, r.predict(x_test)))

### Step 6: Analyzing feature importance

Using the get_feature_importance and plot_feature_importance functions from the evolutionary_forest.utils module, we can visualize the importance of each feature in the dataset. This can provide valuable insights into which features contribute the most to the model's predictions.

In [15]:
from evolutionary_forest.utils import get_feature_importance, plot_feature_importance

code_importance_dict = get_feature_importance(r)
plot_feature_importance(code_importance_dict)

### Step 7: Synthesizing new features

In this section, we highlight the ability of the Evolutionary Forest to synthesize new features while maintaining the output format as a Pandas DataFrame. This is particularly useful for keeping track of the newly created high-order features and their relationships with the original features.

When using the feature_append function, it generates new features based on the top features identified earlier and appends them to the original DataFrame. The output is a Pandas DataFrame with column names that represent the high-order features, clearly indicating their relationship with the initial features. This makes it easy to interpret and understand the newly generated features and their potential impact on the model's performance.

By maintaining the output as a Pandas DataFrame and providing meaningful column names, the Evolutionary Forest algorithm ensures a seamless integration with your existing data processing pipeline and allows you to further analyze and manipulate the synthesized features using the familiar and powerful Pandas library.

In [16]:
from evolutionary_forest.utils import feature_append

# Synthesize new features
code_importance_dict = get_feature_importance(r, simple_version=False)
top_features = list(code_importance_dict.keys())[:len(code_importance_dict) // 2]
new_train = feature_append(r, pd.DataFrame(r.x_scaler.transform(x_train), columns=x_train.columns), top_features,
                           only_new_features=False)
new_test = feature_append(r, pd.DataFrame(r.x_scaler.transform(x_train), columns=x_train.columns), top_features,
                          only_new_features=False)
new_train

### Conclusion
In this notebook, we demonstrated how the Evolutionary Forest Regressor can be used with Pandas DataFrames, making it a powerful tool for working with complex data structures. By showcasing its ability to generate insights, such as feature importance, and synthesize new features, we hope to inspire you to try out this versatile algorithm in your own data science projects.
